# Working with Surface Obs

This notebook creates a colored temperature plot for North America using AWIPS METAR observations (datatype *obs*), similar to existing products in GEMPAK and CAVE.  
We will be accessing the data through an EDEX server and limiting what's returned based geographic extend, and specific parameters we're interested in.  We'll also be creating a color mapping and colorizing the surface data based on our mapping.  Finally we use matplotlib to plot and display the output.

---
## Import necessary packages

In this example we'll need a few packages to assist us when working with the data.  So, the first thing we need to do is set our imports accordingly.

In [ ]:
from awips.dataaccess import DataAccessLayer
from dynamicserialize.dstypes.com.raytheon.uf.common.time import TimeRange
from datetime import datetime, timedelta
import numpy as np
import cartopy.crs as ccrs
import warnings
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from metpy.plots import StationPlot

---
## Define our 'request' object

If you read through the "python-awips - How to Access Data" training, you will know that we need to set an EDEX url to access our server, and then we create a data request.  In the previous training, we simply used the 'datatype' to define our request, but in this example we will also use an 'envelope.  Feel free to [take a look at the documentation](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.newDataRequest) for more information about what arguments can passed through when creating a new data request.

Once the request is made, we'll also restrict it by setting the parameters of interest, in this case just: "temperature", "longitute", "latitude", and "stationName".

In [ ]:
# CONUS bounding box and envelope geometry
bbox=[-120, -70, 15, 55]
envelope = Polygon([(bbox[0],bbox[2]),(bbox[0],bbox[3]),
                    (bbox[1], bbox[3]),(bbox[1],bbox[2]),
                    (bbox[0],bbox[2])])

# New obs request
edexServer = "edex-cloud.unidata.ucar.edu"
DataAccessLayer.changeEDEXHost(edexServer)
request = DataAccessLayer.newDataRequest("obs", envelope=envelope)

# Set the parameters we care about
params = ["temperature", "longitude", "latitude", "stationName"]
request.setParameters(*(params))

# Take a look at our request
print(request)

---
## Limit results based on time

Let's narrow down our results by creating a time window for the most recent 15 minutes.
> Note: You can play around with changing the `time_limit` to see how many observations you get back.

In [ ]:
# Get records from the last 15 minutes
time_limit = 15
recentDateTime = datetime.utcnow() - timedelta(minutes = time_limit)
start = recentDateTime.strftime('%Y-%m-%d %H:%M:%S')
end = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

beginRange = datetime.strptime( start , "%Y-%m-%d %H:%M:%S")
endRange = datetime.strptime( end , "%Y-%m-%d %H:%M:%S")
timerange = TimeRange(beginRange, endRange)

---
## Get the data response

Now we're going to access the data using [***DataAccessLayer.getGeometryData()***](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getGeometryData) and passing in the time range we just defined.

In [ ]:
# Get response
response = DataAccessLayer.getGeometryData(request,timerange)
# Use the response to get the Metar data
obs = DataAccessLayer.getMetarObs(response)

# See how much data we got
print("Found " + str(len(response)) + " total records")
print("Using " + str(len(obs['temperature'])) + " temperature records")

---
## Define color thresholds

We're going to be plotting the station temperature values on a map of the US, and we'll colorize them based on their value.  So, here we define a dictionary defining temperature ranges and their corresponding colors.

In [ ]:
# A list of colors for the temperatures
colors = ['purple', 'c', 'royalblue', 'darkgreen', 'green', 'y', 'orange', 'r']

# A list of the temperature values defining the thresholds
#  Note: these need to be the same length!
levels = [0, 10, 20, 30, 40, 50, 60, 70]

---
## Create a plot to draw on

Here we create a plot use [***matplotlib.pyplot***](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.html) (see the imports at the top of this notebook for reference), and we set our extend using the boundaries we defined in the first second code cell of this notebook.  We also add the coastlines and a title to the plot.

In [ ]:
# Create a plot to draw on
fig, ax = plt.subplots(figsize=(16,12),subplot_kw=dict(projection=ccrs.LambertConformal()))

# Use the same bounding box we used on the data request
ax.set_extent(bbox)

# Draw the coastlines 
ax.coastlines(resolution='50m')

# Set a title
ax.set_title(str(response[-1].getDataTime()) + " | Surface Temps (degF) | " + edexServer)

---
## Colorize and draw the temperatures


In [ ]:
# Suppress nan masking warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

# get all temperature values and convert them from C to F
tair = np.array(obs['temperature'], dtype=float)
tair[tair == -9999.0] = 'nan'
tair = (tair*1.8)+32

# get all the lats and lons for all the observations
lats = obs['latitude']
lons = obs['longitude']

# cycle through each temperature range and plot the corresponding data as appropriate
for i, temp in enumerate(levels):
    # create a new temperature value array
    subtair = tair.copy()

    # pair down the temperature values to a subset
    # if this is the maximum value, then find values greater than it
    if temp==max(levels):
        subtair[(subtair < temp)] = 'nan'
    # if this is the minimum value, then find values less than or equal to it
    elif temp==min(levels):
        subtair[(subtair >= temp)] = 'nan'
    # otherwise find values between it and the next value
    else:
        subtair[(subtair < temp)] = 'nan'
        subtair[(subtair >= levels[i+1])] = 'nan'

    # add these stations and their color to the stationplots
    stationplot = StationPlot(ax, lons, lats, transform=ccrs.PlateCarree(), fontsize=14)
    stationplot.plot_parameter('C', subtair, color=colors[i])

# Display the plot with the colorized temperatures
fig